In [3]:
import tensorflow as tf

### Constant

In [10]:
const = tf.constant('Hello World!')

### Placeholder

Se inicializa vacio, a diferencia de Constant y Variable que parten con un valor

In [12]:
place_1 = tf.placeholder(tf.int16)
place_2 = tf.placeholder(tf.int16)

In [13]:
add = tf.add(place_1, place_2)

### Session

In [16]:
with tf.Session() as sess:
    print(sess.run(const))
    print(sess.run(add, feed_dict={place_1: 2, place_2: 3}))

b'Hello World!'
5


## Deep Learning

La idea de una red neuronal es similar a la de una regresion lineal, tenemos y = Wx + b, donde



\begin{align}
W &= Weight \\ b &= bias \\ x &= data \\ y &= neuron
\end{align}

Donde ademas cada neurona recibe la multiplicacion de cada neurona de la capa anterior por un peso + bias y realiza una función de activación


## Linear Neuron
<img src="https://jalammar.github.io/images/NNs_bias_2.png",width=500,height=500>

## Multilayer Perceptron

<img src="http://www.mdpi.com/information/information-03-00756/article_deploy/html/images/information-03-00756-g002-1024.png",width=500,height=500>

### Multiplicacion de matrices

En el siguiente ejemplo:

**mat_1** es una matriz de 1x3 que representa a 3pixeles de una imagen

**mat_2** es una matriz de 3x4 corresponde a la matriz de pesos 

Vamos a considerar un input de 3 pixeles que van a una hidden layer de 4 neuronas, entonces cada pixel se multiplicara por 4 "pesos" distintos, uno para cada cada neurona de la hidden layer, cada neurona suma los resultados de la multiplicación + el bias, y le aplica la función de activación

**El resultado** de la mutliplicación de **mat_1** y **mat_2** dara origen a la primera hidden layer, que es una capa de 1x4 neuronas

In [32]:
import numpy as np

In [47]:
mat_1 = np.matrix([1,2,3])
mat_2 = np.matrix([[1,2,3,4], [5,6,7,8], [9,10,11,12]])

In [53]:
print("Pixeles      ", mat_1.shape)
print("W            ", mat_2.shape)
print("Pixeles x W  ", mat_1.dot(mat_2).shape)

Pixeles       (1, 3)
W             (3, 4)
Pixeles x W   (1, 4)


In [54]:
mat_1.dot(mat_2) 

matrix([[38, 44, 50, 56]])

### MLP en Tensorflow

In [ ]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [4]:
# Parameters

learning_rate = 0.01
n_epochs = 15
batch_size = 100

In [5]:
# Layers

n_hidden_1 = 256
n_hidden_2 = 256
input_layer = 784
n_classes = 10
x = tf.placeholder("float", [None, input_layer])
y = tf.placeholder("float", [None, n_classes])

In [6]:
# Weights and bias

weights = {
    'w_layer1': tf.Variable(tf.random_normal([input_layer, n_hidden_1])),      # random_normal([dim, cantidad])
    'w_layer2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'output': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
    
}

bias = {
    'b_layer1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b_layer2': tf.Variable(tf.random_normal([n_hidden_2])),
    'output': tf.Variable(tf.random_normal([n_classes]))
}

In [7]:
def multilayer_perceptron(x, W, b):
    
    # Hidden layer 1
    layer_1 = tf.add(tf.matmul(x, W['w_layer1']), b['b_layer1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Hidden layer 2
    layer_2 = tf.add(tf.matmul(layer_1, W['w_layer2']), b['b_layer2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Output
    output = tf.add(tf.matmul(layer_2, W['output']), b['output'])
    
    return output

In [12]:
# Nuestra red se encargara de predecir los pesos, pero debemos ajustarlos mediante la cost function o loss function
# que representa la diferencia entre el valor real y el valor predecido (una forma de representar el error),
# para eso utilizaremos un optimizador que se encargará de minimizar esta función de costo (es decir, minimizar el error)

# Una medida de error al ser entrenada con un modelo para disminuirla se transforma en loss/cost function

pred = multilayer_perceptron(x, weights, bias)

cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [13]:
init = tf.initialize_all_variables()

In [ ]:
with tf.Session as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        
        # Por cada epoca calculamos la cantidad de batches que vamos a extraer de la data
        avg = 0
        number_of_batches = int(mnist.train.num_examples/batch_size)
        
        for batch in range(number_of_batches):
            
            # Por cada batch vamos a sacar esos trozos de data 
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # y corremos el algoritmos, por cada batch recalcula el costo
            _, c= sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            avg_cost += c / total_batch
            
            # Al final de la epoca podemos mostrar el avg_cost calculado en la epoca


In [ ]:
# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})